In [ ]:
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods
import matplotlib as mpl
import matplotlib.pyplot as plt
import max_functions as mf
%matplotlib inline
import mplhep as hep
plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS
import seaborn as sns

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

file = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CL_11800-1400ns_user_LUT.root")
file["angular_distr_el/ID3_mol_e0_valid/EN_gate"].keys()
dict(file["angular_distr_el/ID3_mol_e0_valid/EN_gate"].classes())
# processid = file["angular_distr_el/ID3_mol_e0_valid/EN_gate/PIPICO"] 
# processid = file["angular_distr_el/ID3_mol_e0_valid/EN_gate/check_cthetaEE_x"]
processid = file["FMOx_m43-69/electrons/hit_0/momenta/p_y vs p_z"]  # test fot the TH2D graphs
processid._members() # alternativ to dir(obj) or obj.__dict__
type(processid) # to check the type of object

In [ ]:
processid.edges;
from pprint import pprint
l = dir(processid)
l1 = dir(processid._fXaxis)
d = processid.__dict__
d1 = processid._fXaxis.__dict__
# pprint(l)
# pprint(d, indent=2)
# pprint(l1)
# pprint(d1, indent=2)
processid._fXaxis._fTitle # from __dict__ one can find
# type(processid._fXaxis._fTitle) # from __dict__ one can find

In [ ]:
pipanda = processid.pandas() # uproot method to convert the object into sorted respect to cos(theta) MultiIndex DataFrame type
# pipanda
pipanda.iloc[0] # silicing methods to check the content of the DataFrame providing the index number as an integer
pipanda.loc[0] # as above, providing the name of the index and column
pipanda.head() # first lines of the DataFrame
pipanda.tail() # last lines
pipanda.shape # dimensions of the DataFrame (ROWS, COLOUMS)
pipanda.size # number of elements of the array
len(pipanda) # number of intems of the first dimension
pipanda.columns # in this case of the type MultiIndex
pipanda.index # in this case of the type MultiIndex
pipanda.index.to_frame() # anoter way to fisualise the MultiIndex

In [ ]:
unpipanda = pipanda.unstack() # unstack a multiindex, but in this case is better to pivot
pivot_pipanda = pipanda.pivot_table(values="count", index=processid._fXaxis._fTitle.decode(), columns=processid._fYaxis._fTitle.decode()) # to avoid any confusion with str() use . decode() from byte-strings
pivot_pipanda.columns # for hostograms created in ROOT, the index of bins are of the type IntervalIndex
pivot_pipanda.columns.left # size = 102 equal to the size of .columns
pivot_pipanda.index # size = 22 equal to both .right and .left
pivot_pipanda.index.get_level_values(0) # identical to .index alone
pivot_pipanda.index._data # another equivalent way to show the values
pivot_pipanda.index.left.isin(["-inf"]) # it creates a Boolean mask to eventally remove inf and -inf
# pivot_pipanda.index.reindex(["-inf",10])

Plotting the C matrix of counts with x and y as INDEX and COLUMN! While using
pcolormesh will complain for mismatch of size and for non integer/float numbers in the index or column

In [ ]:
# creates an numpy.ndarray with left + right members
# append DOES NOT CREATE ARRAYS OF UNIQUE NUMBERS, but the InterIndex is already right excluded (i.e. no repetitions)
xbinedges = np.append(pivot_pipanda.columns.left, pivot_pipanda.columns.right) # size = 204
ybinedges = np.append(pivot_pipanda.index.left, pivot_pipanda.index.right) # size = 22

# inf and -inf are the left and right values of the tuples

X, Y = np.meshgrid(xbinedges, ybinedges)
print(X.shape) # (44,204)
print(pivot_pipanda.values.shape) # (22,102) note values.T to traspose the matrix
# plt.pcolormesh(X, Y, pivot_pipanda.values) # inf values are not recognised

NUMPY APPROACH: SEAMS THE RIGHT ONE TO PLOT USING HEP LIBRARY

In [ ]:
pinumpy = processid.numpy() # since pandas is built on top of numpy this return the same format of data, but it is not possible to pivot
z = pinumpy[0] # ndarray .shape = (20,100)
x = pinumpy[1][0][0] # ndarray len(x) = 21
y = pinumpy[1][0][1] # ndarray len(y) = 101

type(pinumpy)

In [ ]:
print(z.shape, x.shape, y.shape)
print(type(x),type(y),type(z))

In [ ]:
cmap = mpl.cm.viridis
cmap.set_under('w')

plt.imshow(pinumpy[0].T) #plots just the values without the right index and column! Has to be transpose to show the correct results
plt.imshow(pinumpy[0].T,extent=[x[0],x[-1],y[0],y[-1]], cmap=cmap, vmin=1.1, aspect="equal") # to plot the right x and y values of the axis
plt.colorbar()

In [ ]:
cmap = mpl.cm.viridis
cmap.set_under('w')

# hep.hist2dplot(pinumpy[0],pinumpy[1][0][0],pinumpy[1][0][1]) # so far the only alternative to plot. It doesn't work with pandas
hep.hist2dplot(z,x,y,vmin=1.) # so far the only alternative to plot. It doesn't work with pandas

In [ ]:
cmap = mpl.cm.viridis
cmap.set_under('w')

x_red=np.array((x[1:] + x[:-1])/2) # from the suggestion of jpivarski: this returns the centre!
y_red=np.array((y[1:] + y[:-1])/2) # from the suggestion of jpivarski: this returns the centre!
plt.contourf(x_red,y_red,z.T, extent=[x[0],x[-1],y[0],y[-1]], cmap=cmap, vmin=1.6)
plt.colorbar()